#02 tratamiento y preparacion datos



En este notebook se realiza el proceso de limpieza, transformación y estandarización de los datos provenientes de las tablas de siniestros, vehículos y actores viales.

El objetivo es consolidar un dataset limpio, consistente y estructuralmente adecuado para etapas posteriores de análisis descriptivo.

## Importación de librerías

In [109]:
import pandas as pd
import numpy as np

##1. Carga de datos

In [110]:
#importar datos
ruta ="https://raw.githubusercontent.com/Josemena95/data-portfolio/refs/heads/main/etl-traffic-accidents-bogota-2023/data/raw/SIGAT_ANUARIO_2023.xlsx"

siniestros = pd.read_excel(ruta)
vehiculos = pd.read_excel(ruta, sheet_name="Vehiculos")
actores = pd.read_excel(ruta, sheet_name="Actor_vial")

## 2. Selección de variables relevantes

Se seleccionan variables relevantes según diccionario de datos.

Se descartan variables no necesarias.

In [111]:
# seleccion de campos a utilizar del Dataframe siniestros
siniestros = siniestros[['Codigo_Accidente','Longitud','Latitud','Fecha_Acc','Hora_Min_Acc','Localidad','Clase_Acc','Gravedad_Indicador_Tradicional',
            'Con_Bicicleta','Con_Carga','Con_Embriaguez','Con_Huecos','Con_Menores','Con_Moto','Con_Peaton','Con_Persona_Mayor','Con_Velocidad','Con_Trans_Public']]

In [112]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14106 entries, 0 to 14105
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Codigo_Accidente                14106 non-null  int64         
 1   Longitud                        14106 non-null  float64       
 2   Latitud                         14106 non-null  float64       
 3   Fecha_Acc                       14106 non-null  datetime64[ns]
 4   Hora_Min_Acc                    14106 non-null  object        
 5   Localidad                       14106 non-null  object        
 6   Clase_Acc                       14106 non-null  object        
 7   Gravedad_Indicador_Tradicional  14106 non-null  object        
 8   Con_Bicicleta                   2149 non-null   object        
 9   Con_Carga                       876 non-null    object        
 10  Con_Embriaguez                  554 non-null    object        
 11  Co

In [113]:
# Seleccion de campos a utilizar del DataFrame vehículos.
vehiculos = vehiculos[['Codigo_Accidente','Codigo_Vehiculo','Clase','Vehiculo_Viajaba_Clasificado']]


In [114]:
vehiculos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24403 entries, 0 to 24402
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Codigo_Accidente              24403 non-null  int64 
 1   Codigo_Vehiculo               24403 non-null  int64 
 2   Clase                         24399 non-null  object
 3   Vehiculo_Viajaba_Clasificado  24403 non-null  object
dtypes: int64(2), object(2)
memory usage: 762.7+ KB


In [115]:
# Seleccion de campos a utilizar del DataFrame actores.
actores = actores[['Codigo_Accidente','Codigo_Vehiculo','Edad','Sexo','Gravedad_Indicador_Tradicional','Muerte_Posterior','Condicion']]

In [116]:
actores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33542 entries, 0 to 33541
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Codigo_Accidente                33542 non-null  int64  
 1   Codigo_Vehiculo                 33542 non-null  int64  
 2   Edad                            32418 non-null  float64
 3   Sexo                            32421 non-null  object 
 4   Gravedad_Indicador_Tradicional  33538 non-null  object 
 5   Muerte_Posterior                33542 non-null  object 
 6   Condicion                       33542 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.8+ MB


##3. Integración de tablas

Se realiza la integración de las tres tablas utilizando como llave primaria el campo Codigo_Accidente y Codigo_Vehiculo, generando un único DataFrame consolidado.

In [117]:
# Relacionar los DataFrames actores, vehículos y siniestros mediante la columna Codigo_Accidente

df = pd.merge(siniestros, vehiculos, on='Codigo_Accidente', how='inner')
df = pd.merge(df, actores, on=('Codigo_Accidente','Codigo_Vehiculo'), how='inner')

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33542 entries, 0 to 33541
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Codigo_Accidente                  33542 non-null  int64         
 1   Longitud                          33542 non-null  float64       
 2   Latitud                           33542 non-null  float64       
 3   Fecha_Acc                         33542 non-null  datetime64[ns]
 4   Hora_Min_Acc                      33542 non-null  object        
 5   Localidad                         33542 non-null  object        
 6   Clase_Acc                         33542 non-null  object        
 7   Gravedad_Indicador_Tradicional_x  33542 non-null  object        
 8   Con_Bicicleta                     4702 non-null   object        
 9   Con_Carga                         2100 non-null   object        
 10  Con_Embriaguez                    1170 non-nul

##4. Tratamiento de valores faltantes

Se realiza imputación determinística en variables binarias, asignando el valor "NO" en los casos donde no se registra información explícita, dado que el dataset únicamente reporta el valor "SI".

Posteriormente, se eliminan registros con información incompleta en variables esenciales para el análisis (Edad, Sexo y clasificación del vehículo).

In [119]:
#Llenado de datos vacíos de acuerdo a la información suministrada por la fuente de los datos.
df['Con_Bicicleta']=df['Con_Bicicleta'].fillna('NO')
df['Con_Carga']=df['Con_Carga'].fillna('NO')
df['Con_Embriaguez']=df['Con_Embriaguez'].fillna('NO')
df['Con_Huecos']=df['Con_Huecos'].fillna('NO')
df['Con_Menores']=df['Con_Menores'].fillna('NO')
df['Con_Moto']=df['Con_Moto'].fillna('NO')
df['Con_Peaton']=df['Con_Peaton'].fillna('NO')
df['Con_Persona_Mayor']=df['Con_Persona_Mayor'].fillna('NO')
df['Con_Velocidad']=df['Con_Velocidad'].fillna('NO')

In [120]:
#eliminacion de registros sin informacion
df = df.dropna(subset=['Fecha_Acc', 'Localidad'])
df =df.dropna(subset=['Sexo', 'Edad']) # Se eliminan los datos que no tienen sexo o edad
df = df[df['Vehiculo_Viajaba_Clasificado'] !='SIN INFORMACIÓN'] # se elimina los datos que estan sin informacion en el campo'Vehiculo_Viajaba_Clasificado'

#Reiniciar el indice
df.reset_index(drop=True, inplace=True)
#Se elimina Gravedad_Indicador_Tradicional_x
df = df.drop('Gravedad_Indicador_Tradicional_x', axis=1)
#df = df.drop('Codigo_Accidente', axis=1)
df = df.drop('Codigo_Vehiculo', axis=1)

###4.1 Corrección de tipos de datos

In [121]:
#Transformacion del campo "Hora_Min_Acc" a formato Hora:Minuto
df['Hora_Min_Acc'] = pd.to_datetime(df['Hora_Min_Acc'], format='%H:%M').dt.time

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32384 entries, 0 to 32383
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Codigo_Accidente                  32384 non-null  int64         
 1   Longitud                          32384 non-null  float64       
 2   Latitud                           32384 non-null  float64       
 3   Fecha_Acc                         32384 non-null  datetime64[ns]
 4   Hora_Min_Acc                      32384 non-null  object        
 5   Localidad                         32384 non-null  object        
 6   Clase_Acc                         32384 non-null  object        
 7   Con_Bicicleta                     32384 non-null  object        
 8   Con_Carga                         32384 non-null  object        
 9   Con_Embriaguez                    32384 non-null  object        
 10  Con_Huecos                        32384 non-nu

##5. Eliminación de duplicados

In [123]:
#Verificación de registros duplicados.
df.loc[df.duplicated()]

,Codigo_Accidente,Longitud,Latitud,Fecha_Acc,Hora_Min_Acc,Localidad,Clase_Acc,Con_Bicicleta,Con_Carga,Con_Embriaguez,...,Con_Persona_Mayor,Con_Velocidad,Con_Trans_Public,Clase,Vehiculo_Viajaba_Clasificado,Edad,Sexo,Gravedad_Indicador_Tradicional_y,Muerte_Posterior,Condicion
254,10597294,-74.118716,4.657884,2023-10-18,08:10:00,FONTIBON,Choque,NO,NO,NO,...,NO,NO,NO,Motocicleta,MOTOCICLETA,28.0,Masculino,HERIDO,N,MOTOCICLISTA
1049,10589402,-74.041753,4.740589,2023-03-29,20:33:00,USAQUEN,Choque,SI,NO,NO,...,NO,NO,SI,Bicicleta,BICICLETA,19.0,Masculino,HERIDO,N,CICLISTA
1330,10593505,-74.107731,4.597793,2023-07-10,10:20:00,ANTONIO NARIÑO,Choque,NO,NO,NO,...,NO,NO,NO,Motocicleta,MOTOCICLETA,28.0,Masculino,HERIDO,N,MOTOCICLISTA
1521,10593563,-74.145412,4.595304,2023-07-13,19:38:00,KENNEDY,Choque,NO,NO,NO,...,SI,NO,SI,Microbus,TRANSPORTE DE PASAJEROS,26.0,Masculino,HERIDO,N,PASAJERO
1523,10593563,-74.145412,4.595304,2023-07-13,19:38:00,KENNEDY,Choque,NO,NO,NO,...,SI,NO,SI,Microbus,TRANSPORTE DE PASAJEROS,53.0,Masculino,HERIDO,N,PASAJERO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31757,10597007,-74.144671,4.588062,2023-10-08,03:00:00,TUNJUELITO,Choque,NO,NO,NO,...,NO,SI,NO,Motocicleta,MOTOCICLETA,19.0,Masculino,HERIDO,N,MOTOCICLISTA
31779,10595319,-74.144856,4.618993,2023-08-27,06:32:00,KENNEDY,Choque,NO,NO,NO,...,NO,NO,NO,Automovil,LIVIANO,20.0,Masculino,HERIDO,N,PASAJERO
31984,10595834,-74.040115,4.760345,2023-09-10,05:09:00,USAQUEN,Caida de ocupante,NO,NO,NO,...,NO,NO,NO,Motocicleta,MOTOCICLETA,17.0,Femenino,HERIDO,N,PASAJERO
32045,10596087,-74.109113,4.506708,2023-09-17,00:55:00,USME,Choque,NO,NO,NO,...,NO,NO,NO,Motocicleta,MOTOCICLETA,24.0,Masculino,HERIDO,N,MOTOCICLISTA


In [124]:
#Eliminación de registros duplicados.
df = df.drop_duplicates()

#Reiniciar el indice
df.reset_index(drop=True,inplace=True)

In [125]:
#Verificación de registros duplicados
df.loc[df.duplicated()]

,Codigo_Accidente,Longitud,Latitud,Fecha_Acc,Hora_Min_Acc,Localidad,Clase_Acc,Con_Bicicleta,Con_Carga,Con_Embriaguez,...,Con_Persona_Mayor,Con_Velocidad,Con_Trans_Public,Clase,Vehiculo_Viajaba_Clasificado,Edad,Sexo,Gravedad_Indicador_Tradicional_y,Muerte_Posterior,Condicion


##6. Enriquecimiento y transformación de variables

###6.1 Reducción de granularidad

Con el fin de reducir la cardinalidad y mejorar la interpretabilidad y consistencia de las categorías, se agrupan categorías con baja frecuencia dentro de clases más generales.


In [126]:
#Debido a la poca frecuencia del valor 'Otro' e 'Incendio' en el campo 'Frecuencias de Clase_Acc', se unen para reducir la cardinalidad en 'Otros'
df['Clase_Acc']=df['Clase_Acc'].replace({'Incendio':'Otros','Otro':'Otros','Volcamiento':'Otros'})


In [127]:
# Se reduce la cardinalidad de 'Vehiculo_Viajaba_Clasificado' agrupando
# En la categoría 'Transporte público', se agrupan los vehículos 'TRANSPORTE DE PASAJEROS' y 'SERVICIO ESPECIAL'
df['Vehiculo_Viajaba_Clasificado']= df['Vehiculo_Viajaba_Clasificado'].replace({'TRANSPORTE DE PASAJEROS':'TRANSPORTE PUBLICO','SERVICIO ESPECIAL':'TRANSPORTE PUBLICO'})

In [128]:
# En la categoría 'Moto', se agrupan 'Motocicleta' y 'Motociclo'
df['Clase']=df['Clase'].replace({'Motocicleta':'Moto','Motociclo':'Moto'})

#En la categoría 'Camión', se agrupan 'Camión', 'Furgón', 'Tractocamión', 'Volqueta', 'Campero' y 'Camioneta'
df['Clase']=df['Clase'].replace({'Camion, Furgon':'Camion','Tractocamion':'Camion','Volqueta':'Camion','Campero':'Camion','Camioneta':'Camion'})

#En la categoría 'Transporte Público', se agrupan 'Bus', 'Microbús' y 'Buseta'
df['Clase']=df['Clase'].replace({'Bus':'Transporte público','Microbus':'Transporte público','Buseta':'Transporte público'})

#En la categoría 'Otros', se agrupan 'No identificado', 'Motocarro', 'M. Industrial', 'M. Agrícola' y 'Tracción animal''
df['Clase']=df['Clase'].replace({'No identificado':'Otros','Motocarro':'Otros','M. Industrial':'Otros','M. Agricola':'Otros','Traccion animal':'Otros','Cuatrimoto':'Otros'})

###6.2 Creación de variables derivadas

Se crea la variable clasificacion_Horario con el objetivo de agrupar la hora del accidente en franjas temporales homogéneas (madrugada, mañana, tarde y noche), facilitando el análisis descriptivo por periodos del día y reduciendo la dispersión de la variable original.

In [129]:
#variables de tiempo
HoraInicioMadrugada=pd.to_datetime('00:00:00', format='%H:%M:%S').time()
HoraFinMadrugada=pd.to_datetime('06:00:00', format='%H:%M:%S').time()
HoraInicioMañana=pd.to_datetime('06:00:00', format='%H:%M:%S').time()
HoraFinMañana=pd.to_datetime('12:00:00', format='%H:%M:%S').time()
HoraInicioTarde=pd.to_datetime('12:00:00', format='%H:%M:%S').time()
HoraFinTarde=pd.to_datetime('18:00:00', format='%H:%M:%S').time()
HoraInicioNoche=pd.to_datetime('18:00:00', format='%H:%M:%S').time()
HoraFinNoche=pd.to_datetime('23:59:59', format='%H:%M:%S').time()

In [130]:
# Funcion de clasificacion de Horaria
def clasificacion_hora(x):
  if x>=HoraInicioMadrugada and x<=HoraFinMadrugada:
    return 'Madrugada'
  elif x>=HoraInicioMañana and x<=HoraFinMañana:
    return 'Mañana'
  elif x>=HoraInicioTarde and x<=HoraFinTarde:
    return 'Tarde'
  elif x>=HoraInicioNoche:
    return 'Noche'
  else:
    return 'No definido'

In [131]:
#creacion de campo 'clasificacion_Horario'
df["clasificacion_Horario"]=df["Hora_Min_Acc"].apply(clasificacion_hora)

##7. Construcción de tablas analíticas

### Tabla acidentados por localidad y fecha

In [132]:
# Agrupar por fecha y localidad y contar el número de personas involucradas en accidentes
df2 = df[['Fecha_Acc','Localidad']]

localidad_por_fecha = df2.groupby(['Fecha_Acc', 'Localidad']).size().unstack(fill_value=0)
localidad_por_fecha['Dia_semana']=localidad_por_fecha.index.day_name()
dia=localidad_por_fecha.pop('Dia_semana')
localidad_por_fecha.insert(0, 'Dia_semana', dia)
localidad_por_fecha.columns.name = None

### Tabla acidentados por fecha

In [133]:
#agrupar por fecha
personas_accidentadas_dia=df2.groupby('Fecha_Acc').size()
personas_accidentadas_dia.name='Personas_accidentadas_por_dia'
personas_accidentadas_dia= personas_accidentadas_dia.reset_index()

#conversion de columna "Fecha_Acc"  a indice
personas_accidentadas_dia.set_index('Fecha_Acc', inplace=True)


## Descripción de los datasets generados



Como resultado del proceso ETL se generan tres datasets con diferentes niveles de granularidad:

### 1. Dataset transaccional limpio
Contiene la información consolidada a nivel de registro individual (persona involucrada en accidente), integrando las tablas de siniestros, vehículos y actores viales.

Incluye variables espaciales, temporales, categóricas y demográficas, ya transformadas y normalizadas. Este dataset constituye la base para análisis descriptivo detallado.

---

### 2. Dataset agregado por localidad y fecha
Tabla analítica agregada que muestra el número de personas accidentadas por día y por localidad.

Permite analizar patrones espaciales y temporales, identificar concentraciones geográficas y evaluar comportamiento semanal mediante la variable `Dia_semana`.

---

### 3. Dataset agregado por fecha
Tabla temporal agregada que contiene el número total de personas accidentadas por día en la ciudad.

Este dataset facilita el análisis de tendencias, estacionalidad semanal y variaciones temporales.

##8. Exportación de datasets

In [134]:
df.to_csv("accidentes_bogota_2023_dataset_limpio.csv",index=False)
localidad_por_fecha.to_csv("accidentes_bogota_2023_localidad_fecha_agregado.csv",index=True)
personas_accidentadas_dia.to_csv("accidentes_bogota_2023_agg_fecha.csv",index=True)

El proceso ETL concluye con la generación de tres datasets estructurados, los cuales serán utilizados en el notebook de análisis descriptivo.
